In [0]:
# pip install snap-stanford

In [0]:
# import snap

#Imports and Data loading

In [0]:
!pip install demon

In [0]:
import json
import pprint

In [0]:
import demon as d

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
tweets =[]
with open('/content/drive/My Drive/Ai_Ethics/Gamergate.json','r') as f:
    for line in f.readlines():
        tweets.append(json.loads(line))

In [0]:
with open('/content/drive/My Drive/Ai_Ethics/Labels_for_7320_users/gamergate_opposer_feminist_usernames', 'r') as f:
    opposer =f.read().splitlines()
with open('/content/drive/My Drive/Ai_Ethics/Labels_for_7320_users/gamergate_supporter_misogynist_usernames','r') as f:
    supporter = f.read().splitlines()
with open('/content/drive/My Drive/Ai_Ethics/Labels_for_7320_users/unaffiliated_usernames','r') as f:
    unaffiliated = f.read().splitlines()

In [0]:
allusers = opposer + unaffiliated + supporter

In [0]:
# type(allusers)

list

#Netowork Graph creation

In [0]:
import networkx as nx

In [0]:
G = nx.Graph()

In [0]:
G.add_nodes_from(allusers)

In [0]:
for tweet in tweets:
    username = tweet['user']['screen_name']
    if username not in allusers:
        continue
    if 'retweeted_status' not in tweet:
        continue
    retweeteduser = tweet['retweeted_status']['user']['screen_name']
    if retweeteduser not in allusers:
        continue
    # G.add_nodes_from([username,retweeteduser])
    G.add_edge(username,retweeteduser)

In [0]:
# len(G.nodes)

In [0]:
G = G.to_undirected()

In [0]:
l0 = list(G.edges)

In [0]:
len(l0)

30238

In [0]:
# import demon as d

#Algorithm

In [0]:
dm = d.Demon(graph=G, epsilon=0.25, min_community_size=3)
coms = dm.execute()

Exec: 100%|████████████████████████

Time:  'execute' 16.26 sec
------------------------------------


In [0]:
# type(coms[0])

tuple

In [0]:
lo = []
for i in coms:
  # print(len(i))
  lo.append(len(i))

In [0]:
m1 = []
m2 = []
m3 = []
for i in coms:
  c1 = 0
  c2 = 0
  c3 = 0
  for j in i:
    if j in opposer:
      c1+=1
    if j in supporter:
      c2+=1
    if j in unaffiliated:
      c3+=1
  m1.append(c1)
  m2.append(c2)
  m3.append(c3)

In [0]:
m1

[288, 170, 298, 303, 287, 359, 357, 284, 524, 363, 298, 624, 415, 292]

In [0]:
m2

[82, 79, 98, 758, 649, 805, 843, 746, 765, 870, 727, 800, 985, 772]

In [0]:
m3

[4, 3, 4, 7, 6, 6, 11, 7, 8, 7, 4, 8, 11, 6]

In [0]:
m4 = []
for i in range(len(m1)):
  mk = [m1[i],m2[i],m3[i]]
  m4.append(mk.index(max(mk)))

In [0]:
m4

[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [0]:
# len(allusers)

7320

In [0]:
act = []
for i in allusers:
  if i in opposer:
    act.append(0)
  if i in supporter:
    act.append(1)
  if i in unaffiliated:
    act.append(2)

In [0]:
len(act)

7320

In [0]:
dict1 = {}
c = 0
li = []
for i in allusers:
  lu = []
  for j in range(len(coms)):
    if i in coms[j]:
      lu.append(m4[j])
  if(len(lu) == 1):
    dict1[i] = lu[0]
  elif(len(lu) > 1):
    dict1[i] = max(set(lu), key = lu.count) 
  elif(len(lu) == 0):
    li.append(i)
  # print(c)
  c+=1

In [0]:
len(li)

4810

#Predictions and Evaluation

In [0]:
act1 = []
pred = []
for i in allusers:
  if i not in li:
    if i in opposer:
      act1.append(0)
    if i in supporter:
      act1.append(1)
    if i in unaffiliated:
      act1.append(2)
    pred.append(dict1[i]) 


In [0]:
len(pred)

2510

In [0]:
discarded_users = len(li)

In [0]:
## precentage discared users

print('presentage discarded users: ',discarded_users*100/len(G.nodes))

presentage discarded users:  65.7103825136612


In [0]:
discarded_ht = []
for tweet in tweets:
    if tweet['user']['screen_name'] in li:
        discarded_ht.extend([ht['text'].lower() for ht in tweet['entities']['hashtags']])
discarded_ht = set(discarded_ht)

In [0]:
# len(discarded_ht)

In [0]:
all_ht = []
for tweet in tweets:
    if tweet['user']['screen_name'] in allusers:
        all_ht.extend([ht['text'].lower() for ht in tweet['entities']['hashtags']])
all_ht = set(all_ht)

In [0]:
# len(all_ht)

1479

In [0]:
## precentage discarded hashtags

print('precentage discarded hashtags: ',len(discarded_ht)*100/len(all_ht))

precentage discarded hashtags:  61.730899256254226


In [0]:
# discarded_users

In [0]:
from sklearn.metrics import classification_report

In [0]:
print(classification_report(act1,pred))

              precision    recall  f1-score   support

           0       0.78      0.31      0.44      1050
           1       0.64      0.94      0.76      1438
           2       0.00      0.00      0.00        22

    accuracy                           0.67      2510
   macro avg       0.47      0.42      0.40      2510
weighted avg       0.70      0.67      0.62      2510



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import jaccard_score

In [0]:
onehot = OneHotEncoder(handle_unknown='ignore')
jaccard_gt = onehot.fit_transform([[i] for i in act1]).toarray()
jaccard_pred = onehot.transform([[i] for i in pred]).toarray()

In [0]:
print('jaccard score :',jaccard_score(jaccard_gt, jaccard_pred, average='samples'))

jaccard score : 0.6673306772908366


In [0]:
# len(dict1.keys())

2510